In [60]:
import qiskit as qk
import numpy as np
import scipy.linalg as lin
import math
from qiskit.quantum_info import Statevector , Operator , partial_trace , DensityMatrix
from qiskit.circuit.library.standard_gates import HGate , XGate
from qiskit.circuit.library import GlobalPhaseGate
from qiskit.extensions import Initialize , UnitaryGate
from qiskit.providers.aer import AerSimulator
import sys
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator
from qiskit import QuantumCircuit, Aer, transpile
from pmr_LCU import *

## Parameters of the system

Here, we define the parameters for the simulation of the Hamiltonian $H(t) = \sum_{i=1}^n h_i Z_i + V_x \sum_{i=1}^{n-1} Z_i Z_{i+1} + cos(\omega t) \sum_{i=1}^{n-1} c_i X_i X_{i+1}$.  

In [61]:
#====================== Circuit Paramters ================
number_of_spins = 2 # number of particles (qubits)
C0 = 1.0 # Parameter for the floquet interaction strength
Coeffs = C0*np.array([2+(-1)**x for x in np.arange(1,number_of_spins)]) # there are n elements in this array
Omega = 1.0
h0 = 1.0
Longit_h = h0*np.array([1.5 + 0.5*(-1)**x for x in range(number_of_spins)])
Vx = 1.0
C = np.max(Coeffs)

Gammas_i = Coeffs
Gammas_k = [0.5 , 0.5]
Qmax = 2

Gamma_1 = np.sum(Gammas_i)
Gamma_2 = np.sum(Gammas_k)
Gamma = Gamma_2 * Gamma_1
Delta_t = np.log(2)/Gamma
GDt = Gamma*Delta_t

In [62]:
K = 2     #only two modes -w and w
Nkq = int(np.log2(K))*Qmax
Niq = int(np.log2(number_of_spins))*Qmax
Ntotal = Nkq + Niq + number_of_spins + 2 + Qmax

kqQubits = qk.QuantumRegister(Nkq , '|kq>')
iqQubits = qk.QuantumRegister(Niq , '|iq>')
zQubits = qk.QuantumRegister(number_of_spins + 1 , '|z>') # n+1 st is an ancilla for diagonal rotations!
ancQubit = qk.QuantumRegister(1 , '|anc>')
qQubits = qk.QuantumRegister(Qmax , '|q>')

kq_qbits_index = 0
iq_qbits_index = 1
z_qbits_index = 2
anc_qbits_index = 3
q_qbits_index = 4

FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , ancQubit , qQubits )

Gammas_list = [Gammas_i , Gammas_k]
# =========================== Building the circuits  U_0(dt) U_od(dt) ... U_0(dt) U_od(dt) =================================== #
FullCirc = qk.QuantumCircuit( kqQubits , iqQubits , zQubits , ancQubit, qQubits )
B_prepare(FullCirc , Qmax , Delta_t , Gammas_list , kq_qbits_index , iq_qbits_index , q_qbits_index ,  dagger=False)
#Prepare_full_unitary( FullCirc , number_of_spins , Qmax , current_time , Gamma_function , kq_qbits_index , iq_qbits_index , z_qbits_index , q_qbit_index , r_number ):

#Ops = SparsePauliOp.from_list([('I'*(Nkq + Niq)+'XIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IXII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'ZIII'+'I'*Q , 1) , ('I'*(Nkq + Niq)+'IZII'+'I'*Q , 1)])

#print(f'Circuit Prepared! The parameters are:  Vx = {Vx} , hs = {Longit_h} , Omega = {Omega} , C0 = {C0} , Q = {Q} , Gamma = {Gam} , Delta_t = {Delta_t} , GDt = {Gam*Delta_t}') 

Mdisc is 0
inside of the permutation states unitary, the vec is [[1.]]
inside of the permutation states unitary, the vec is [[1.]]


/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/qiskit_0.42/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/qiskit_0.42/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/pmr_LCU.py:95: SyntaxWarning: invalid escape sequence '\o'
  """
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/pmr_LCU.py:272: SyntaxWarning: invalid escape sequence '\o'
  """"
/Users/armanbabakhani/Dropbox/Research/Itay/QuantumCircuits/Qiskit/Quantum_Circuit_Github/Quantum_Circuits_Floquet/pmr_LCU.py:301: SyntaxWarning: invalid escape sequence

CircuitError: 'The amount of qubit(1)/clbit(0) arguments does not match the gate expectation (0).'

In [162]:
# Transpiling the circuit and getting a gate count:
backend_sim = Aer.get_backend('qasm_simulator')
transpiled_circuit = transpile(FullCirc, backend_sim)

gate_count = transpiled_circuit.count_ops()
gate_depth = transpiled_circuit.depth()
Toffoli_count = gate_count.get('ccx' , 0)
CNOT_count = gate_count.get('cx' , 0)
#print("\nGate Count:", gate_count)
print(f'The gate depth is {gate_depth} , total number of qubits required is {Ntotal}, the number of Toffoli gates is {Toffoli_count}, and number of CNOTs is {CNOT_count}')

#print(FullCirc.draw())

The gate depth is 308 , total number of qubits required is 10, the number of Toffoli gates is 36, and number of CNOTs is 158


In [163]:
IBMProvider.save_account(token='1a6f8188ff1654f78d9c4418c7382addf037bf1aca1ec8a40fe866c40123a30913609310721808e16cf25f3f5f92de14f163d30a8162b5a0bba7bda1324ca1cc', overwrite=True )
provider = IBMProvider(instance="usc/hen-research-g/hen-lab")

backend = provider.get_backend('ibm_strasbourg')
Estimator = BackendEstimator(backend)
job = Estimator.run( [FullCirc]*len(Ops) , [Ops[i] for i in range(len(Ops))]  , shots = 5000 )
id=job.job_id()
print(f'The job id is {id}')

The job id is 3978659f-249a-4f61-bd27-78e21a3ca017


In [164]:
results = job.result()
#counts = results.get_counts()


In [165]:
values = results.values
metadata = results.metadata
print(f'The values are {values} and the metadata is {metadata}')

The values are [ 0.0176  0.138   0.942  -0.1512] and the metadata is [{'variance': 0.99969024, 'shots': 5000}, {'variance': 0.980956, 'shots': 5000}, {'variance': 0.11263600000000007, 'shots': 5000}, {'variance': 0.97713856, 'shots': 5000}]
